In [1]:
#RANDOM FOREST MODEL

import numpy as np
import pandas as pd
from matplotlib.dates import MonthLocator, DateFormatter
import joblib
from sklearn.ensemble import RandomForestRegressor

mytrainingdata = pd.read_csv('train_data_augmented.csv',index_col=0) #added index
mytestdata = pd.read_csv('test_data_augmented.csv',index_col=0) #added index


#The input features
X_train = mytrainingdata.drop(['Max_Demand_GW','Plant_Production_GWh'],axis = 1)
y_train = mytrainingdata['Max_Demand_GW']
y_test = mytestdata['Max_Demand_GW']
X_test = mytestdata.drop(['Max_Demand_GW','Plant_Production_GWh'],axis = 1)


# Random Forest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,explained_variance_score,r2_score

#Instantiate the model
model = RandomForestRegressor(n_estimators = 70,bootstrap = False, max_depth = 10, min_samples_leaf = 1, min_samples_split = 2, random_state = 30)

#n_estimators = number of trees
#random_state = random seed for reproducibility

#Fitting the rf model to the training data
fitted_rf = model.fit(X_train, y_train)



#EXPORT MODEL AS JOBLIB

# Save the model to a file using joblib
joblib.dump(model, 'random_forest_model.joblib')

# Now you can load the model later using the following:
loaded_model = joblib.load('random_forest_model.joblib')

# Now 'loaded_model' is ready to be used in Gradio or for making predictions


In [5]:
#GRADIO

import gradio as gr
import joblib
import numpy as np
import csv

# Load pre-trained models
random_forest_model = joblib.load('random_forest_model.joblib')
#decision_tree_model = joblib.load('decision_tree_model.joblib')
#svm_model = joblib.load('svm_model.joblib')
#linear_regression_model = joblib.load('linear_regression_model.joblib')
#neural_network_model = joblib.load('neural_network_model.joblib')

# Read examples from CSV
csv_file = 'test_data_augmented.csv'

with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    data_from_csv = [row[3:] for row in reader]

# Add the last column as 'Random Forest' NEED TO CHANGE THIS ONCE OTHER MODELS ARE ADDED
data_with_model_type = [row + ['Random Forest'] for row in data_from_csv]


# Function to generate prediction
def generate_prediction(imports, renewables_production, powerplants_c02, gdp, population,tmin,tavg,tmax,model_type):
    # Perform prediction using the selected model_type
    if model_type == 'Random Forest':
        model = random_forest_model
    #elif model_type == 'Decision Tree':
    #   model = decision_tree_model
    #elif model_type == 'SVM':
        #model = svm_model
    #elif model_type == 'Linear Regression':
        #model = linear_regression_model
    #elif model_type == 'Neural Network':
        #model = neural_network_model
    else:
        model = None  # Default
    
    if model is not None:
        input_data = np.array([[imports, renewables_production, powerplants_c02, gdp, population,tmin,tavg,tmax]])
        prediction = model.predict(input_data)[0]
    else:
        prediction = 0  # Default

    # Return the prediction as a number
    return prediction
    

# Gradio interface
iface = gr.Interface(
    fn=generate_prediction,
    inputs=[
        "number","number", "number", "number", "number", "number", "number", "number",
        gr.Dropdown(choices=['Random Forest'], label="Select Model")
    ],
    outputs="number",
    live=True,
    title="Max Demand Prediction",
    description="Enter features and select model to get the predicted Max Demand.",
    examples=data_with_model_type
)

# Launch the Gradio interface
iface.launch()


Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


- User can choose options from the examples - which are all the test data

- For now outputting max demand as a number instead of a graph. WILL NEED TO ADD THE DATE TO THE EXAMPLES TO BE ABLE TO SHOW THEM ON THE GRAPH. What date would the prediction be? Shall we ask for date input?



Work to do:
    
- CAN SHOW THE OUTPUTS FOR DIFFERENT MODELS UNDER EACH OTHER


- Need the inputs each model works with - e.g. mine needs all features, what do the others need

- Need to get joblibs for each model.

- Finalise interface - add sliders, maybe template, etc

- ADD VALIDATION FOR USER INPUT

